# Oscilloskope utility – using Ethernet with GPS timemarks 

In [28]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11
import serial
import struct
import math

# pyUblox libraries
import ublox
import util
import datetime

gpsport = '/dev/tty.usbmodem1411'
gpsbaudrate = 115200

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run this cell to verify the connection.

# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
            
# For Ethernet
#osc = TmcDriver("TCPIP::147.231.24.72::INSTR")
osc = TmcDriver("TCPIP::10.1.1.254::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::10.1.1.254::INSTR
RIGOL TECHNOLOGIES,DS2102A,DS2D174501960,00.03.04.SP2


In [12]:
secsInWeek = 604800 
secsInDay = 86400 
gpsEpoch = (1980, 1, 6, 0, 0, 0)  # (year, month, day, hh, mm, ss) 

def UTCFromGps(gpsWeek, SOW, leapSecs=14): 
      """converts gps week and seconds to UTC 
   
      see comments of inverse function! 
   
      SOW = seconds of week 
      gpsWeek is the full number (not modulo 1024) 
      """ 
      secFract = SOW % 1 
      epochTuple = gpsEpoch + (-1, -1, 0)  
      t0 = time.mktime(epochTuple) - time.timezone  #mktime is localtime, correct for UTC 
      tdiff = (gpsWeek * secsInWeek) + SOW - leapSecs 
      t = t0 + tdiff 
      (year, month, day, hh, mm, ss, dayOfWeek, julianDay, daylightsaving) = time.gmtime(t) 
      #use gmtime since localtime does not allow to switch off daylighsavings correction!!! 
      return (year, month, day, hh, mm, ss + secFract) 

def parsegps(gpsdata):
    for msg in re.findall('\xb5b\r.{33}', gpsdata):
        (h1,h2,h3,h4,hl) = struct.unpack('<BBBBH', msg[0:6])
        (chan,flags) = struct.unpack('<BB',msg[6:8])
        print(hex(h1), hex(h2), hex(h3), hex(h4),hl)
        (count, wnR, wnF, towMsR, towSubMsR, towMsF, towSubMsF, accEst) = \
        struct.unpack('<HHHIIIII', msg[8:34])
        print(count,wnR, towMsR)
        print UTCFromGps(wnR, towMsR/1000, towSubMsR/1000000)


## Read repeatedly records from oscilloscope

In [ ]:
filename = 0

if (filename == 1):
    for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
        print 'Deleting', f
        os.remove(f)
else:
    print 'Not removing old files, as filename {0} is not 1.'.format(filename)

# Initialize GPS grabber
print 'Initializing GPS grabber'
dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=1)
dev.set_logfile(None, append=True)
dev.set_binary()
dev.configure_poll_port()
dev.configure_poll(ublox.CLASS_CFG, ublox.MSG_CFG_USB)
dev.configure_poll(ublox.CLASS_MON, ublox.MSG_MON_HW)
dev.configure_port(port=ublox.PORT_SERIAL1, inMask=1, outMask=0)
dev.configure_port(port=ublox.PORT_USB, inMask=1, outMask=1)
dev.configure_port(port=ublox.PORT_SERIAL2, inMask=1, outMask=0)
dev.configure_poll_port()
dev.configure_poll_port(ublox.PORT_SERIAL1)
dev.configure_poll_port(ublox.PORT_SERIAL2)
dev.configure_poll_port(ublox.PORT_USB)
dev.configure_solution_rate(rate_ms=100)

dev.set_preferred_dynamic_model(ublox.DYNAMIC_MODEL_STATIONARY)
dev.set_preferred_usePPP(None)

dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_POSLLH, 0)
dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_POSECEF, 0)
dev.configure_message_rate(ublox.CLASS_RXM, ublox.MSG_RXM_RAW, 0)
dev.configure_message_rate(ublox.CLASS_RXM, ublox.MSG_RXM_SFRB, 0)

# Poll UTC time
dev.configure_message_rate(ublox.CLASS_NAV, ublox.MSG_NAV_TIMEUTC, 0)
# Poll TM2 messages
dev.configure_message_rate(ublox.CLASS_TIM, ublox.MSG_TIM_TM2,1)



while True:
    # Start the run
    print 'Issuing START command'
    osc.write(':START') # Stop button
    time.sleep(0.5)
    
    print 'Waiting for TM2 message'
    zero_count = None
    
    # Wait for one pulse
    while True:
        msg = dev.receive_message()
        if (msg is None) and (zero_count is None):
            print 'Reopening.'
            dev.close()
            dev = ublox.UBlox(gpsport, baudrate=gpsbaudrate, timeout=1)
            dev.set_logfile(None, append=True)
        elif (msg is None):
            break
        else:
            try:
                #print(str(msg))
                sys.stdout.flush()
            except ublox.UBloxError as e:
                print(e)
            if msg.name() == 'TIM_TM2':
                #print('Got TM2 message')
                try:
                    msg.unpack()
                    if (zero_count is None):
                        zero_count = msg.count - 1
                        last_count = msg.count
                        first_towMsR = msg.towMsR
                        count_base = 0;
                        print('First counter value: ' + str(msg.count))
                    if (msg.count < zero_count+1):
                        count_base += 65535;
                    timestring = datetime.datetime.utcfromtimestamp(util.gpsTimeToTime(msg.wnR, 1.0e-3*msg.towMsR))
                    txt = "Counter = %5d Zero = %5d Diff = %5d %s\n" % (msg.count-zero_count, zero_count, msg.count-last_count, timestring)
                    print txt
                    last_count = msg.count
                except ublox.UBloxError as e:
                    print(e)

    print '  TM2 counter initialized from GPS.'
    
    # Stop the oscilloscope
    osc.write(':STOP') # Stop button
    time.sleep(0.5)    
    
    osc.write(':FUNC:WREC:OPER REC') # start recording
    run_start_time = time.time()
    print '  Capturing data from oscilloscope...'
    time.sleep(0.5)
    
    while True:
        osc.write(':FUNC:WREC:OPER?') # finished recording?
        reply = osc.read()
        if reply == 'STOP':
            run_time = round(time.time() - run_start_time, 2)
            print('  Subrun finished after %.2f seconds.' % run_time)
            dev.close()
            break
        time.sleep(0.01)

    # Data readout
    osc.write(':WAV:SOUR CHAN1')
    osc.write(':WAV:MODE NORM')
    osc.write(':WAV:FORM BYTE')
    osc.write(':WAV:POIN 1400')

    osc.write(':WAV:XINC?')
    xinc = float(osc.read(100))
    print 'XINC:', xinc,
    osc.write(':WAV:YINC?')
    yinc = float(osc.read(100))
    print 'YINC:', yinc,
    osc.write(':TRIGger:EDGe:LEVel?')
    trig = float(osc.read(100))
    print 'TRIG:', trig,
    osc.write(':WAVeform:YORigin?')
    yorig = float(osc.read(100))
    print 'YORIGIN:', yorig,
    osc.write(':WAVeform:XORigin?')
    xorig = float(osc.read(100))
    print 'XORIGIN:', xorig,
    
    osc.write(':FUNC:WREP:FEND?') # get number of last frame
    frames = int(osc.read(100))
    print 'FRAMES:', frames, 'SUBRUN', filename
    
    with h5py.File('./data/data'+'{:02.0f}'.format(filename)+'_'+str(int(round(run_start_time,0)))+'.h5', 'w') as hf:       
        hf.create_dataset('FRAMES', data=(frames)) # write number of frames
        hf.create_dataset('XINC', data=(xinc)) # write axis parameters
        hf.create_dataset('YINC', data=(yinc))
        hf.create_dataset('TRIG', data=(trig))
        hf.create_dataset('YORIGIN', data=(yorig))
        hf.create_dataset('XORIGIN', data=(xorig))
        hf.create_dataset('CAPTURING', data=(run_time))
        osc.write(':FUNC:WREP:FCUR 1') # skip to n-th frame
        time.sleep(0.5)
        for n in range(1,frames+1):
            osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
            time.sleep(0.001)

            osc.write(':WAV:DATA?') # read data
            #time.sleep(0.4)
            wave1 = bytearray(osc.read_raw(500))
            wave2 = bytearray(osc.read_raw(500))
            wave3 = bytearray(osc.read_raw(500))
            #wave4 = bytearray(osc.read(500))
            #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
            wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
            hf.create_dataset(str(n), data=wave)

    # Increase filename prefix
    filename = filename + 1
    

Not removing old files, as filename 0 is not 1.
Initializing GPS grabber
try set binary at 115200
Issuing START command
Waiting for TM2 message
MON_HW INVALID_SIZE1=60
Reopening.
Reopening.
TIM_TM2: ch=0, flags=245, count=4215, wnR=1983, wnF=1983, towMsR=30902495, towSubMsR=114451, towMsF=30902495, towSubMsF=99368, accEst=26
First counter value: 4215
Counter =     1 Zero =  4214 Diff =     0 2018-01-07 08:35:02.495000

TIM_TM2: ch=0, flags=245, count=4226, wnR=1983, wnF=1983, towMsR=30902596, towSubMsR=782454, towMsF=30902596, towSubMsF=771183, accEst=26
Counter =    12 Zero =  4214 Diff =    11 2018-01-07 08:35:02.596000

TIM_TM2: ch=0, flags=117, count=4246, wnR=1983, wnF=1983, towMsR=30902798, towSubMsR=944856, towMsF=30902808, towSubMsF=164376, accEst=26
Counter =    32 Zero =  4214 Diff =    20 2018-01-07 08:35:02.798000

  TM2 counter initialized from GPS.
  Capturing data from oscilloscope...


In [12]:
from ipywidgets import widgets
from IPython.display import display

def button_clicked(b):
    print 'Trigger bigger' + str(b)

start_button = widgets.Button(description='Start capture')
stop_button = widgets.Button(description='Stop capture')
hbox = widgets.HBox([start_button, stop_button])

start_button.on_click(button_clicked)
stop_button.on_click(button_clicked)

display(hbox)

HBox(children=(Button(description=u'Start capture', style=ButtonStyle()), Button(description=u'Stop capture', style=ButtonStyle())))

Trigger biggerButton(description=u'Stop capture', style=ButtonStyle())
Trigger biggerButton(description=u'Start capture', style=ButtonStyle())
Trigger biggerButton(description=u'Stop capture', style=ButtonStyle())
